<font color="#138D75">**NERO Winter School training**</font> <br>
**Copyright:** (c) 2025 EUMETSAT <br>
**License:** GPL-3.0-or-later <br>
**Authors:** Andrea Meraner (EUMETSAT), based on [HLS data sources example](https://github.com/nasa/HLS-Data-Resources/blob/main/python/tutorials/HLS_Tutorial.ipynb).
# Access and process Landsat 8/9 and Sentinel-2 30-m imagery from Earthdata cloud HLS datasets


## Intro

The Harmonized Landsat and Sentinel-2 (HLS) NASA project takes input data from the joint NASA/USGS Landsat 8 and Landsat 9 and the ESA (European Space Agency) Sentinel-2A, Sentinel-2B, and Sentinel-2C satellites to generate a harmonized, analysis-ready surface reflectance data product with observations every two to three days depending on the geographical region at 30m spatial resolution. The aim is to produce seamless products with normalized parameters, which include atmospheric correction, cloud and cloud-shadow masking, geographic co-registration and common gridding, normalized bidirectional reflectance distribution function, and spectral band adjustment.

Two data products are generated as part of the HLS project: the L30 data product generated with Landsat 8 and Landsat 9 data, and the S30 product generated using Sentinel-2 data. These data are available through Earthdata Search as well as through NASA's Land Processes Distributed Active Archive Center (LP DAAC).

In this notebook, we will access and process data from Landsat 8/) and Sentinel-2 data using the Earthdata python API and cloud-based access using rioxarray. NASA's Land Processes Distributed Active Archive Center (LP DAAC) archives and distributes HLS products in the LP DAAC Cumulus cloud archive as Cloud Optimized GeoTIFFs (COG). Because these data are stored as COGs, this notebook shows how to load subsets of individual files into memory for just the bands you are interested in--a paradigm shift from the more common workflow where you would need to download a .zip/HDF file containing every band over the entire scene/tile.

## Learning Objectives

- How to work with HLS Landsat ([HLSL30.002](https://doi.org/10.5067/HLS/HLSL30.002)) and Sentinel-2 ([HLSS30.002](https://doi.org/10.5067/HLS/HLSS30.002)) data products
- How to query and subset HLS data using the `earthaccess` library
- How to access and work with HLS data

## Links and Resouces
- HLS project page: https://www.earthdata.nasa.gov/data/projects/hls
- HLS LP-DAAC overview with useful tables and technical information:  https://lpdaac.usgs.gov/data/get-started-data/collection-overview/missions/harmonized-landsat-sentinel-2-hls-overview/
- HLS resources Github page with code examples (this notebook is heavily based on these): https://github.com/nasa/HLS-Data-Resources/
- HLS User Guide: https://lpdaac.usgs.gov/documents/1698/HLS_User_Guide_V2.pdf
- Earthaccess documentation: https://earthaccess.readthedocs.io/en/latest/


<hr>


## Before we start
To run this script you will need to configure your access to the Earthdata API. You have to first get your username and password by registering at https://urs.earthdata.nasa.gov/ , you will then be prompted to insert them by the script.

<hr>

### Setup
Let's start by importing all required modules and setting some GDAL configuration options required for the correct reading of the remote data.

In [1]:
import os
import sys
import pyproj

pyproj.datadir.set_data_dir(os.path.join(sys.prefix, 'share', 'proj'))
import earthaccess
import geopandas as gpd
import pandas as pd
import rioxarray as rxr
from osgeo import gdal
from shapely.geometry import Polygon
import hvplot.xarray
import hvplot.pandas

import credentials

# GDAL configurations used to successfully access LP DAAC Cloud Assets via vsicurl
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE', '~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN', 'EMPTY_DIR')
# keep also shapefile extensions to not disturb the lsasaf script
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS', 'TIF,SHP,SHX,DBF,PRJ')
gdal.SetConfigOption('GDAL_HTTP_UNSAFESSL', 'YES')
gdal.SetConfigOption('GDAL_HTTP_MAX_RETRY', '10')
gdal.SetConfigOption('GDAL_HTTP_RETRY_DELAY', '0.5')

Let's login to the Earthdata for our future queries:

In [2]:
earthaccess.login(persist=True)

Configure your query by defining the time and area. The times should be in the format "2024-09-16T00:00:00" and will indicate the start time and end time of the search for Landsat/Sentinel2 granules. The latitude-longitude bounding box `lonlat_bbox` is generated from the `W`est, `S`outh, `E`ast, `N`orth variables.

In [3]:
start_time = "2024-09-15T00:00:00"
end_time = "2024-09-20T23:59:59"

#  lat-lon geographical bounds of search area
W = 26.5
S = 41.7
E = 27.3
N = 42.3
lonlat_bbox = [W, S, E, N]

run_name = "testrun"

output_dir = './testdir/'

### Search data
Now let's build and execute the query to earthaccess

In [4]:
results = earthaccess.search_data(
    short_name=['HLSL30', 'HLSS30'],
    bounding_box=tuple(lonlat_bbox),
    temporal=(start_time, end_time),
)

We can preview these results in a `pandas` `dataframe`. Note we only show the first 5. If the query returns empty, try to widen your time and/or area query. Each result represents a so-called "granule", a section of the satellite acquisitions along the orbits.

In [5]:
pd.json_normalize(results).head(5)

,size,meta.concept-type,meta.concept-id,meta.revision-id,meta.native-id,meta.collection-concept-id,meta.provider-id,meta.format,meta.revision-date,umm.TemporalExtent.RangeDateTime.BeginningDateTime,...,umm.CollectionReference.EntryTitle,umm.RelatedUrls,umm.DataGranule.DayNightFlag,umm.DataGranule.Identifiers,umm.DataGranule.ProductionDateTime,umm.DataGranule.ArchiveAndDistributionInformation,umm.Platforms,umm.MetadataSpecification.URL,umm.MetadataSpecification.Name,umm.MetadataSpecification.Version
0,204.958004,granule,G3240742384-LPCLOUD,1,HLS.S30.T29TNF.2024259T112111.v2.0,C2021957295-LPCLOUD,LPCLOUD,application/echo10+xml,2024-09-17T01:40:09.573Z,2024-09-15T11:29:59.462Z,...,HLS Sentinel-2 Multi-spectral Instrument Surfa...,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,Day,[{'Identifier': 'HLS.S30.T29TNF.2024259T112111...,2024-09-17T01:37:48.000Z,"[{'Name': 'Not provided', 'SizeInBytes': 21491...","[{'ShortName': 'Sentinel-2A', 'Instruments': [...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,1.6.6
1,211.419896,granule,G3240736359-LPCLOUD,1,HLS.S30.T29TNE.2024259T112111.v2.0,C2021957295-LPCLOUD,LPCLOUD,application/echo10+xml,2024-09-17T01:32:10.888Z,2024-09-15T11:30:14.079Z,...,HLS Sentinel-2 Multi-spectral Instrument Surfa...,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,Day,[{'Identifier': 'HLS.S30.T29TNE.2024259T112111...,2024-09-17T01:28:48.000Z,"[{'Name': 'Not provided', 'SizeInBytes': 22168...","[{'ShortName': 'Sentinel-2A', 'Instruments': [...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,1.6.6
2,129.538170,granule,G3246325661-LPCLOUD,1,HLS.S30.T29TNF.2024262T113321.v2.0,C2021957295-LPCLOUD,LPCLOUD,application/echo10+xml,2024-09-23T07:43:04.038Z,2024-09-18T11:39:56.856Z,...,HLS Sentinel-2 Multi-spectral Instrument Surfa...,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,Day,[{'Identifier': 'HLS.S30.T29TNF.2024262T113321...,2024-09-23T07:41:01.000Z,"[{'Name': 'Not provided', 'SizeInBytes': 13583...","[{'ShortName': 'Sentinel-2A', 'Instruments': [...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,1.6.6
3,83.451330,granule,G3246308534-LPCLOUD,1,HLS.S30.T29TNE.2024262T113321.v2.0,C2021957295-LPCLOUD,LPCLOUD,application/echo10+xml,2024-09-23T07:17:06.602Z,2024-09-18T11:40:11.246Z,...,HLS Sentinel-2 Multi-spectral Instrument Surfa...,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,Day,[{'Identifier': 'HLS.S30.T29TNE.2024262T113321...,2024-09-23T07:13:55.000Z,"[{'Name': 'Not provided', 'SizeInBytes': 87505...","[{'ShortName': 'Sentinel-2A', 'Instruments': [...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,1.6.6
4,242.675860,granule,G3247355819-LPCLOUD,1,HLS.S30.T29TNF.2024264T112109.v2.0,C2021957295-LPCLOUD,LPCLOUD,application/echo10+xml,2024-09-24T07:07:06.265Z,2024-09-20T11:29:55.766Z,...,HLS Sentinel-2 Multi-spectral Instrument Surfa...,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,Day,[{'Identifier': 'HLS.S30.T29TNF.2024264T112109...,2024-09-24T07:04:16.000Z,"[{'Name': 'Not provided', 'SizeInBytes': 25446...","[{'ShortName': 'Sentinel-2B', 'Instruments': [...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,1.6.6


We can  preview each individual granule by selecting it from the results list. This will show the data links, and a browse image. Pick your favourite as we will continue to process this specific granule.

In [6]:
granule = results[0]
granule

Collection: {'EntryTitle': 'HLS Sentinel-2 Multi-spectral Instrument Surface Reflectance Daily Global 30m v2.0'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -7.70336265, 'Latitude': 40.55547314}, {'Longitude': -7.68379957, 'Latitude': 41.54431153}, {'Longitude': -9.00023982, 'Latitude': 41.55184467}, {'Longitude': -9.00023625, 'Latitude': 40.56274991}, {'Longitude': -7.70336265, 'Latitude': 40.55547314}]}}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2024-09-15T11:29:59.462Z', 'EndingDateTime': '2024-09-15T11:29:59.462Z'}}
Size(MB): 204.95800399780273
Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B03.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B12.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B8A.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B01.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.VAA.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B09.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B02.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.SZA.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B05.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B08.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B10.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B11.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B04.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.SAA.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B06.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B07.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.VZA.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.Fmask.tif']

We now need to extract the URLs to each individual spectral band from our result

In [7]:
granule_bands_urls = granule.data_links()
granule_bands_urls

['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B03.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B12.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B8A.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B01.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.VAA.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B09.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30

### Prepare bands for composite
Now it's time to define the bands we want to put in the output composites. Let's define Red-Green-Blue lists for both Landsat and Sentinel data. Refer to [this page](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/missions/harmonized-landsat-sentinel-2-hls-overview/#hls-spectral-bands) for a table and numbering of the respective channels.

The example provided below is the "SWIR"-composite.

In [8]:
bands_sentinel = ["B12", "B8A", "B04"]
bands_landsat = ["B07", "B05", "B04"]
composite_name = "SWIR-composite"

Now let's check whether our picked granule is from Sentinel-2 or Landsat 8/9 and assign the bands accordingly. We then extract the links for the bands that we need for our composite from the list of all bands.

In [9]:
if granule_bands_urls[0].split('/')[4] == 'HLSS30.020':
    instrument_name = 'Sentinel2'
    bands = bands_sentinel

elif granule_bands_urls[0].split('/')[4] == 'HLSL30.020':
    instrument_name = 'Landsat89'
    bands = bands_landsat

band_links = []
for a in granule_bands_urls:
    if any(b in a for b in bands):
        band_links.append(a)

Now let's open the bands we need into an `xarray`. **Note:** if you get errors during this operation, most likely your authentication and configuration for the Earthdata access was not successful. Check your `~/.netrc` file for the correct user and password.

In [10]:
# Use vsicurl to load the data directly into memory (be patient, may take a few seconds)
chunk_size = dict(band=1, x=512, y=512)  # Tiles have 1 band and are divided into 512x512 pixel chunks
for e in band_links:
    print(e)
    if e.rsplit('.', 2)[-2] == bands[0]:
        red_beam = rxr.open_rasterio(e, chunks=chunk_size, masked=True).squeeze('band', drop=True)
        red_beam.attrs['scale_factor'] = 0.0001  # hard coded the scale_factor attribute
    elif e.rsplit('.', 2)[-2] == bands[1]:
        green_beam = rxr.open_rasterio(e, chunks=chunk_size, masked=True).squeeze('band', drop=True)
        green_beam.attrs['scale_factor'] = 0.0001  # hard coded the scale_factor attribute
    elif e.rsplit('.', 2)[-2] == bands[2]:
        blue_beam = rxr.open_rasterio(e, chunks=chunk_size, masked=True).squeeze('band', drop=True)
        blue_beam.attrs['scale_factor'] = 0.0001  # hard coded the scale_factor attribute

https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B12.tif
https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B8A.tif
https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T29TNF.2024259T112111.v2.0/HLS.S30.T29TNF.2024259T112111.v2.0.B04.tif


We can take a quick look at one of the `dataarray` we just read in.

In [11]:
red_beam

<xarray.DataArray (y: 3660, x: 3660)> Size: 54MB
dask.array<getitem, shape=(3660, 3660), dtype=float32, chunksize=(512, 512), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 29kB 5e+05 5e+05 5.001e+05 ... 6.097e+05 6.098e+05
  * y            (y) float64 29kB 4.6e+06 4.6e+06 4.6e+06 ... 4.49e+06 4.49e+06
    spatial_ref  int64 8B 0
Attributes: (12/41)
    ACCODE:                                            LaSRC
    add_offset:                                        0.0
    arop_ave_xshift(meters):                           0
    arop_ave_yshift(meters):                           0
    arop_ncp:                                          0
    arop_rmse(meters):                                 0
    ...                                                ...
    spatial_coverage:                                  99
    SPATIAL_RESOLUTION:                                30
    TILE_ID:                                           S2A_OPER_MSI_L1C_TL_2A...
    ULX:                                               499980
    ULY:                                               4600020
    AREA_OR_POINT:                                     Area

Now that we retrieved the data for the bands we need, let's crop the granule bands to the specific lat-lon bounding box we defined at the beginning. To do that, we need to convert the lat-lon box to the [projection used by HLS](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/missions/harmonized-landsat-sentinel-2-hls-overview/#hls-tiling-system), namely UTM (aligned to the Military Grid Reference System). Since UTM is a zonal projection, we'll need to extract the unique UTM zonal projection parameters from our input HLS files and use them to transform the coordinates of our input area.
We then crop the data using rasterio `clip`.

In [12]:
# Create a Polygon representing the bounding box
min_lon, min_lat, max_lon, max_lat = lonlat_bbox
bounding_box = Polygon([
    (min_lon, min_lat),  # Bottom-left
    (min_lon, max_lat),  # Top-left
    (max_lon, max_lat),  # Top-right
    (max_lon, min_lat),  # Bottom-right
    (min_lon, min_lat)  # Closing the polygon
])
# Create a GeoDataFrame with the bounding box
geo_df = gpd.GeoDataFrame({'geometry': [bounding_box]}, crs="EPSG:4326")  # WGS84 coordinate system

# convert our geo_df of our latlon bounding box to the CRS of the data
fsUTM = geo_df.to_crs(red_beam.spatial_ref.crs_wkt)

# Crop to our ROI
red_beam_cropped = red_beam.rio.clip(fsUTM.geometry.values, fsUTM.crs, all_touched=True)
green_beam_cropped = green_beam.rio.clip(fsUTM.geometry.values, fsUTM.crs, all_touched=True)
blue_beam_cropped = blue_beam.rio.clip(fsUTM.geometry.values, fsUTM.crs, all_touched=True)

Our final band preparation step is the conversion from counts to physical quantities using the band own `scale_factor`. This is a common way for data providers to reduce the size of stored data (at cost of a small quantisation error). For that, we define a scaling function and apply it to all bands.

In [13]:
# Define function to scale
def scaling(band):
    scale_factor = band.attrs['scale_factor']
    band_out = band.copy()
    band_out.data = band.data * scale_factor
    band_out.attrs['scale_factor'] = 1
    return (band_out)


# Apply Scaling
red_beam_cropped_scaled = scaling(red_beam_cropped)
green_beam_cropped_scaled = scaling(green_beam_cropped)
blue_beam_cropped_scaled = scaling(blue_beam_cropped)

Now let's look at one of the bands

In [14]:
red_beam_cropped_scaled
red_beam_cropped_scaled.hvplot.image(cmap='viridis', frame_width=800, fontscale=1.6, crs='EPSG:32610',
                                     tiles=True).opts(
    title='HLS Cropped Red Beam Band')  # Quick visual to assure that it worked

:Overlay
   .WMTS.I  :WMTS   [Longitude,Latitude]
   .Image.I :Image   [x,y]   (SWIR2)

/tcenas/home/andream/anaconda3/envs/devenv/lib/python3.11/site-packages/dask/array/chunk.py:278: RuntimeWarning: invalid value encountered in cast
  return x.astype(astype_dtype, **kwargs)
/tcenas/home/andream/anaconda3/envs/devenv/lib/python3.11/site-packages/dask/array/chunk.py:278: RuntimeWarning: invalid value encountered in cast
  return x.astype(astype_dtype, **kwargs)
/tcenas/home/andream/anaconda3/envs/devenv/lib/python3.11/site-packages/dask/array/chunk.py:278: RuntimeWarning: invalid value encountered in cast
  return x.astype(astype_dtype, **kwargs)
/tcenas/home/andream/anaconda3/envs/devenv/lib/python3.11/site-packages/dask/array/chunk.py:278: RuntimeWarning: invalid value encountered in cast
  return x.astype(astype_dtype, **kwargs)
/tcenas/home/andream/anaconda3/envs/devenv/lib/python3.11/site-packages/dask/array/chunk.py:278: RuntimeWarning: invalid value encountered in cast
  return x.astype(astype_dtype, **kwargs)


Above, you can see that the data have been loaded into memory as subset to our ROI. Note that the array size is considerably smaller than the full size we read in before.


### Generate composite from bands
The final step is now to compose the RGB-composite using the three prepared bands. For this, we use the `get_rgb_array` preconfigured function. The function will rescale our bands to 0-255 values range (8-bit images), and concatenate the red-green-blue arrays into one 3-d array.

In [ ]:
from landsat_sentinel2_img_earthdata_script import get_rgb_array, main_earthdata

rgb = get_rgb_array(red_beam_cropped_scaled, green_beam_cropped_scaled, blue_beam_cropped_scaled)
rgb

In [ ]:
rgb.hvplot.rgb(x='x', y='y', bands='band', frame_width=800, fontscale=1.6, crs='EPSG:32610', tiles=True).opts(
    title='HLS Cropped RGB Composite')  # Quick visual to assure that it worked

Now let's save the array to file.

In [ ]:
from landsat_sentinel2_img_earthdata_script import compute_date

# prepare filename
datestr = granule_bands_urls[0].split('/')[-1].split('.')[3].split('T')
tile_id = granule_bands_urls[0].split('/')[-1].split('.')[2]
# use compute_date to convert from Day-of-Year to date
output_name = os.path.join(output_dir, run_name, 'Satellite_Imagery', 'Landsat-Sentinel2',
                           f"{compute_date(int(datestr[0][0:4]), int(datestr[0][-3:]))}T{datestr[1]}_{composite_name}_{instrument_name}_{tile_id}_cropped.tif")

os.makedirs(os.path.dirname(output_name), exist_ok=True)
rgb.rio.to_raster(raster_path=output_name, driver='COG')

## Process several composites for all granules automatically (optional)
Configure and run the cell below to generate automatically several composites for all granules in the search query. The composites are now configured in a dictionary with the composite name as key and the bands definition for each instrument as value, see example below.

In [ ]:
composites_dict = {
    'SWIR-composite': {
        'Sentinel2': ['B12', 'B8A', 'B04'],
        'Landsat89': ['B07', 'B05', 'B04']
    }
}

main_earthdata(start_time, end_time, lonlat_bbox, output_dir, run_name, composites_dict)